In [1]:
#pip install findspark
#/usr/local/opt/apache-spark/libexec/sbin
#./start-thriftserver.sh

In [2]:
import findspark
findspark.init()
#findspark.add_packages("io.delta:delta-core_2.11:0.1.0")

In [3]:
import pyspark
from pyspark.sql import SparkSession
#from delta import *

spark = SparkSession.builder.appName("spark")\
.config('spark.driver.memory', '32g')\
.config("hive.server2.thrift.port", 10000)\
.config("spark.sql.hive.thriftServer.singleSession", True)\
.config("spark.driver.maxResultSize", "3g")\
.enableHiveSupport()\
.getOrCreate()


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/14 14:06:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
from pyspark.ml.fpm import FPGrowth

In [5]:
data = spark.read.format('parquet').load("/Users/minoseah629/Repos/AIT614Project/ewin/spark-warehouse/ait614projectpermanenttable")
data.createOrReplaceTempView("dataset")
spark.sql("select * from dataset").show()

+---+-----+----------+------+-------------------+-------+--------------------+--------------------+--------------------+--------------------+----------+---------+--------------------+
| ID|Speed|TravelTime|Status|           timedate| LinkId|          LinkPoints|   EncodedLinkPoints| EncodedPolyLineLvls|               Owner|TranscomId|  Borough|           Link_Name|
+---+-----+----------+------+-------------------+-------+--------------------+--------------------+--------------------+--------------------+----------+---------+--------------------+
|330|16.77|       294|     0|2021-12-07 17:58:03|4329507|40.75719,-73.9972...|mkwwFvqsbMsQbh@oI...|                BBBB|  PA -Lincoln Tunnel|   4329507|Manhattan|LINCOLN TUNNEL W ...|
|212|45.98|       236|     0|2021-12-07 17:58:03|4362244|40.78802,-73.7900...|cl}wFtbkaMfFiIdBy...|BBBBBBBBBBBBBBBBB...|   NYC-DOT-Region 10|   4362244|   Queens|CVE NB LIE - WILL...|
|205|41.63|       384|     0|2021-12-07 17:58:03|4362247|40.78819,-73.7905...|em

In [6]:
columns = 'hour(timedate) `hour`,minute(timedate) `minute`,second(timedate) `second`,weekday(timedate) `weekday`,year(timedate) `year`,month(timedate) `month`,day(timedate) `day`,dayofyear(timedate) as `day_of_year`'

In [16]:
df3 = spark.sql('SELECT year(timedate) `year`, dayofyear(timedate) as `day_of_year`, hour(timedate) `hour`,minute(timedate) `minute`, LINKID, avg(SPEED) as `speed` FROM dataset GROUP BY year(timedate), dayofyear(timedate), hour(timedate), minute(timedate), LINKID HAVING avg(SPEED) < 20')
#select id, linkid,speed, traveltime, owner, borough, '+columns+',count(*) as `count1` from dataset group by id, linkid,speed, traveltime, owner, borough,timedate')
display(df3.count())


15840678

In [20]:
from pyspark.sql.window import Window
import random 
from  pyspark.sql.functions import col,array,row_number
df4 = df3.select(df3.LINKID, array(df3.LINKID, df3.speed,  df3.year, df3.hour, df3.minute, df3.day_of_year).alias('items'))
df4.show()

+-------+--------------------+
| LINKID|               items|
+-------+--------------------+
|4616282|[4616282.0, 0.0, ...|
|4456510|[4456510.0, 16.77...|
|4616312|[4616312.0, 13.67...|
|4616341|[4616341.0, 18.01...|
|4616323|[4616323.0, 19.26...|
|4616340|[4616340.0, 12.42...|
|4616282|[4616282.0, 0.0, ...|
|4616298|[4616298.0, 14.29...|
|4456502|[4456502.0, 0.0, ...|
|4620314|[4620314.0, 19.26...|
|4616228|[4616228.0, 0.0, ...|
|4456494|[4456494.0, 0.0, ...|
|4616324|[4616324.0, 3.1, ...|
|4616228|[4616228.0, 0.0, ...|
|4620331|[4620331.0, 9.32,...|
|4616323|[4616323.0, 18.01...|
|4616217|[4616217.0, 0.0, ...|
|4616357|[4616357.0, 11.18...|
|4616309|[4616309.0, 8.07,...|
|4616228|[4616228.0, 0.0, ...|
+-------+--------------------+
only showing top 20 rows



In [21]:
train, test = df4.distinct().randomSplit([0.7,0.3],seed=42)

In [22]:
fpGrowth = FPGrowth(itemsCol="items", minSupport=0.5,minConfidence=0.5)
model = fpGrowth.fit(train)

In [14]:
spark.stop()